In [ ]:
import os
import json
import folium
from sqlalchemy import create_engine
from geoalchemy2 import Geometry
from geoalchemy2.functions import ST_BdMPolyFromText, ST_PolygonFromText
from geoalchemy2.elements import WKTElement
from llama_index import GPTSQLStructStoreIndex, SQLDatabase
from llama_index.indices.struct_store import SQLContextContainerBuilder
from langchain import OpenAI

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
DATABASE_NAME = os.getenv("DATABASE_NAME")
DATABASE_USER = os.getenv("DATABASE_USER")
DATABASE_PASSWORD = os.getenv("DATABASE_PASSWORD")
DATABASE_PORT = os.getenv("DATABASE_PORT")
DATABASE_HOST = os.getenv("DATABASE_HOST")

In [ ]:
llm_predictor = OpenAI(model_name="gpt-4")

In [ ]:
connection_string = f"postgresql://{DATABASE_USER}:{DATABASE_PASSWORD}@{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}"
engine = create_engine(connection_string)


In [ ]:

db = SQLDatabase(engine, include_tables=["stand_4326"])

In [ ]:
stand_4326_text = (
    "The PostGis plugin is available in the database.\n"
    "The forest polygons can be found from the geometry column.\n"
    "Get the query as geojson."
)

table_context_dict = {
    "stand_4326": stand_4326_text,
}
context_builder = SQLContextContainerBuilder(db, context_dict=table_context_dict)
context_container = context_builder.build_context_container()

In [ ]:
index = GPTSQLStructStoreIndex.from_documents(
    [],
    sql_database=db,
    table_name="stand_4326",
    sql_context_container=context_container,
)

In [ ]:
res = index.query("Get the forest polygon with largest area?")

In [ ]:
geojson = json.loads(res.extra_info["result"][0][0])

In [ ]:
type(geojson)

In [ ]:
print(res.extra_info['sql_query'])

In [ ]:
g = geojson["coordinates"][0][0][0]
loc = [round(g[1],2), round(g[0],2)]
m = folium.Map(location=loc,zoom_start=12)
loc

In [ ]:
folium.GeoJson(geojson).add_to(m)

In [ ]:
m